In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

/home/tangzichen/miniconda3/envs/fedllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-06 09:11:17,764] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## 加载基础模型

In [2]:
model = AutoModelForCausalLM.from_pretrained("/home/tangzichen/models/Qwen1.5-0.5B-Chat")
tokenizer = AutoTokenizer.from_pretrained("/home/tangzichen/models/Qwen1.5-0.5B-Chat")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 加载Lora模型

In [3]:
p_model = PeftModel.from_pretrained(model, model_id="/home/tangzichen/OpenFedLLM/output/ShenNong_TCM_Dataset_0_fedavg_c8s4_i-1_b8a1_l512_r32a64_20241105201826/checkpoint-5000")
p_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): Linear(
                in_features=1024, out_features=1024, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(
      

In [5]:
ipt = tokenizer("Human: {}\n{}".format("舌头边缘锯齿状怎么回事，请从中医角度回答", "").strip() + "\n\nAssistant: ", return_tensors="pt")
# tokenizer.decode(p_model.generate(**ipt, do_sample=False)[0], skip_special_tokens=True)
tokenizer.decode(p_model.generate(**ipt, max_length=512)[0])

'Human: 舌头边缘锯齿状怎么回事，请从中医角度回答\n\nAssistant: 从中医的角度来看，舌唇边缘的锯齿状可能是由于口腔溃疡、口腔炎、口干等引起的。在中医中，常见的病因包括肝火上扰、胃热伤阴、气血不足等。建议您到医院或中医诊所进行详细的诊断和治疗。同时，注意口腔卫生，避免辛辣刺激性食物和饮料，保持口腔湿润，多喝水，适当休息，有助于缓解症状。<|im_end|>'

In [6]:
ipt = tokenizer("Human: {}\n{}".format("舌头边缘锯齿状怎么回事，请从中医角度回答", "").strip() + "\n\nAssistant: ", return_tensors="pt")
# tokenizer.decode(p_model.generate(**ipt, do_sample=False)[0], skip_special_tokens=True)
tokenizer.decode(model.generate(**ipt, max_length=512)[0])

'Human: 舌头边缘锯齿状怎么回事，请从中医角度回答\n\nAssistant: 从中医的角度来看，舌头上出现锯齿状的现象，通常是因为体内湿气较重、气血不畅或者脾胃虚弱等因素所致。建议您可以多食用一些清淡易消化的食物，如粥类、蔬菜水果等；同时保持良好的生活习惯，避免过度劳累和情绪波动。如果情况较为严重，建议您咨询专业的中医师进行诊断和治疗。<|im_end|>'

: 

## 模型合并

In [10]:
merge_model = p_model.merge_and_unload()
merge_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(i

In [11]:
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt")
tokenizer.decode(merge_model.generate(**ipt, do_sample=False)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 1. 熟悉考试大纲'

## 完整模型保存

In [9]:
merge_model.save_pretrained("./chatbot/merge_model")